<a href="https://www.kaggle.com/code/rupanshirana/car-s-ex-showroom-price-india-2022?scriptVersionId=123058611" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Objective and Problem Statement
- The main aim of this Notebook is to predict the price of cars using various Machine Learning (ML) models.
- Examine Underlying Data Insights which enables the buyers to purchase the car based on some factors and from business point of view which variables are significant in predicting the price of a car and How well those variables describe the price of a car.

#### DataSource: https://www.kaggle.com/datasets/harmeetsingh07/exshowroom-price
- This dataset contains the number of specifications in a new car like height-weight-length, suspension, body, fuel type, Mileage,audio system, etc everything in a brand new car and then based on all those features it's ex-showroom price is given

- This Dataset contains Columns: 141 entries dtypes: float64(6), int64(1), object(134)

## Workflow
- Step 1: Basic Data understanding
- Step 2: Data cleaning and Data Feature Engineering
- Step 3: Data Insights
- Step 4: Data preparation
- Step 5: Model building and evaluation
- Step 6 : Conclusion

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


## Step 1: Basic Data understanding

##### 1.1 Importing required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,mean_squared_error,r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor

import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option("display.max_columns",500)

##### 1.2 Loading the Data Set

In [ ]:
df=pd.read_csv("/kaggle/input/exshowroom-price/cars_engage_2022 (1).csv")

##### 1.3 Checking shape

In [ ]:
df.shape

##### 1.4 Preview of Data

In [ ]:
df.head()

##### 1.5 Drop irrelevant columns

In [ ]:
df.drop(columns=["Unnamed: 0"],inplace=True)

In [ ]:
df.shape

##### 1.6 Basic understanding of Data

In [ ]:
df.info()  ## too many columns

##### 1.7 Fetching column names

In [ ]:
df.columns

#### ...... is called elipses

## Step 2 : Data Cleaning and Feature Engineering

##### 2.1 Checking missing values

In [ ]:
df.isnull().sum()

- Data contains missing values.

##### 2.2 Checking missing %

In [ ]:
for i in df.columns:
    if df[i].isnull().sum()>0:
        print(i,"----------",df[i].isnull().sum()*100/df.shape[0])

##### 2.3 Deleting the columns having missing % > 70

In [ ]:
for i in df.columns:
    if df[i].isnull().sum()*100/df.shape[0]>70:
        df.drop(columns=[i],inplace=True)

In [ ]:
df.shape

- Now , column count is 115.

##### 2.4 Checking column types  in order to handle the missing values accordingly

In [ ]:
df.dtypes

#### Seggregating the numeric and object data so that we can work on these accordingly.

##### 2.5 Numeric Data

In [ ]:
numeric=["int32","float64"]
df_numeric=df.select_dtypes(include=numeric)

In [ ]:
df_numeric

##### 2.5.1 Filling the missing values of numeric data

In [ ]:
df_numeric.isnull().sum()

##### 2.5.2 Filling missing values of df_numeric with median since median is robust to outliers.

In [ ]:
for i in df_numeric:
    if df_numeric[i].isnull().sum()>0:
        df_numeric[i]=df_numeric[i].fillna(df_numeric[i].median())

In [ ]:
df_numeric.isnull().sum().sum()

- Missing values have been filled in the numeric columns.

##### 2.6 Categorical data

#### Now we will be working on columns of object types.

###### df_obj is dataframe having  columns values of  object type only.

In [ ]:
df_obj=df.select_dtypes(include="object")
df_obj.head()

###### 2.6.1 Extracting important categorical features with respect to price in **df_obj_list**
- Since column count of object data type is 110 (large) so we will fetch those columns which are important with respect to car price which we found through domain knowledge.

In [ ]:
df_obj_list = ['Make', 'Model', 'Variant', 'Ex-Showroom_Price', 'Displacement', 'Drivetrain', 'Emission_Norm','Keyless_Entry',
       'Power', 'Torque', 'Odometer', 'Speedometer', 'Tachometer', 'Tripmeter','Fuel_Type', 'ARAI_Certified_Mileage','Front_Suspension', 'Rear_Suspension','Ground_Clearance','Boot_Space',
 'Audiosystem','Third_Row_AC_Vents', 'Ventilation_System','Auto-Dimming_Rear-View_Mirror', 'Hill_Assist', 'Gear_Indicator','ABS_(Anti-lock_Braking_System)','EBD_(Electronic_Brake-force_Distribution)',
 'Gear_Shift_Reminder','Adjustable_Steering_Column', 'Parking_Assistance', 'Key_Off_Reminder','USB_Compatibility',
 'EBA_(Electronic_Brake_Assist)', 'Seat_Height_Adjustment','Navigation_System', 'Second_Row_AC_Vents', 'Rear_Center_Armrest','ESP_(Electronic_Stability_Program)','Cooled_Glove_Box', 'Turbocharger',
  'Rain_Sensing_Wipers','Automatic_Headlamps', 'ASR_/_Traction_Control', 'Cruise_Control']
len(df_obj_list)

- Column count is 45 of important features.

###### 2.6.2 Since we consider above columns and fetch the data corresponds to them and save it to df_obj

In [ ]:
df_obj=df_obj.loc[:,df_obj_list]

In [ ]:
len(df_obj.columns)

In [ ]:
df_obj.head()

###### 2.6.3 Now , Checking the missing values % in df_obj which is having 45 columns

In [ ]:
for i in df_obj.columns:
    print(i,"---------",(df_obj[i].isnull().sum()/df_obj.shape[0]))

##### 2.6.4 Ambiguity in Model column(Wagon)

In [ ]:
df_obj[df_obj["Model"]=="Wagon"].head()

#### So from above output we can see the ambuiguity in make and model columns with respect to land rover rover and maruti suzuki r and range, wagon in make and model respectively so we will replace land rover rover in make with land rover and maruti suzuki r with maruti suzuki in make column and replace range with range rover and wagon with wagon r in model column.

##### Changes 
- Wagon----Wagon R
- Maruti Suzuki R------Maruti Suzuki
- Land rover rover------ land rover

In [ ]:
df_obj["Make"]=df_obj["Make"].replace("Maruti Suzuki R","Maruti Suzuki")
df_obj["Make"]=df_obj["Make"].replace("Land Rover Rover","Land Rover")

In [ ]:
df_obj["Model"]=df_obj["Model"].replace("Range","Range Rover")
df_obj["Model"]=df_obj["Model"].replace("Wagon","Wagon R")

##### 2.6.5 We will handle all the columns of object type one by one

##### 1 Make

#### Filling the missing values in make column

In [ ]:
df_obj[df_obj["Make"].isnull()][["Make","Model"]]

In [ ]:
'''index=0
for i in df_obj["Model"]:
    f=i[0:3]
    if type(df_obj["Make"].iloc[index])==float:
        if f=="Mer":
            df_obj["Make"].iloc[index]="Mercedes"
        if f=="Rol":
            df_obj["Make"].iloc[index]="Rolls-Royce"
        if f=="Go+":
            df_obj["Make"].iloc[index]="Datsun"
    index=index+1
'''

### OR

##### So mercedes,rolls royce and go+ cars which has corresponding company name is not available so we fill the company name of mercedes,rolls royce and go+ cars(models) corresponding to null place in Make column

###### We use combine_first method to fill null values of Make with the corresponding entry of model then we replace mercedes , rolls-royce , go+ cars with their company name Mercedes, Rolls-Royce and Datsun respectively

- Note : combine_first() method is used to combine two series into one. The result is union of the two series that is in case of Null value in caller series, the value from passed series is taken. In case of both null values at the same index, null is returned at that index.

In [ ]:
df_obj["Make"]=df_obj["Make"].combine_first(df_obj["Model"])
df_obj["Make"]

In [ ]:
#df_obj["Make"].unique()

##### Check missing values again

In [ ]:
df_obj["Make"].isnull().sum()

- NaN values have been filled with their corresponding model's name.

##### Replacing mercedes , rolls-royce , go+  cars with their company name Mercedes, Rolls-Royce and Datsun respectively.

In [ ]:
for i in df_obj["Make"]:
    if "Mercedes" in i:
        df_obj["Make"]=df_obj["Make"].replace(i,"Mercedes-Benz")
    elif "Rolls" in i:
        df_obj["Make"]=df_obj["Make"].replace(i,"Rolls-Royce")
    elif "Go" in i:
        df_obj["Make"]=df_obj["Make"].replace(i,"Datsun")

In [ ]:
#df_obj["Make"].unique()

#### 2 Model does not have missing values
#### 3 Variant doesn't have missng values

##### 4 Ex-Showroom_Price ----  doesn't have missing values

In [ ]:
df["Ex-Showroom_Price"].unique()

##### Removing "Rs. " ,  "," in Ex-Showroom_price as below :

In [ ]:
df_obj["Ex-Showroom_Price"]=df_obj["Ex-Showroom_Price"].str.replace("Rs. ","").str.replace(",","")

In [ ]:
df_obj["Ex-Showroom_Price"].unique()

##### Renaming Ex-Showroom_Price column name to Price

In [ ]:
df_obj.rename(columns={"Ex-Showroom_Price":"Price"},inplace=True)

##### 5 Displacement

##### Checking missing values in Displacement

In [ ]:
df_obj["Displacement"].isnull().sum()

##### Removing " cc" from displacement values :

In [ ]:
df_obj["Displacement"]=df_obj["Displacement"].str.replace(" cc","")

In [ ]:
#df_obj["Displacement"].unique()

#### Checking mode values of all columns of df_obj dataframe

In [ ]:
df_obj["Displacement"].mode()[0]

In [ ]:
for i in df_obj.columns:
    print(i,"----",df_obj[i].mode()[0])

##### Replacing those features having yes or no values with its mode value.

In [ ]:
for i in df_obj.columns:
    if df_obj[i].isna().sum()>0:
        if "Yes" in df_obj[i].unique():
            df_obj[i].fillna("No",inplace = True)
        else:
            df_obj[i].fillna(df_obj[i].mode()[0],inplace=True)

##### Now checking missing values of df_obj again.

In [ ]:
c=0
for i in df_obj.columns:
        print(i,":::::::::::",(df_obj[i].isnull().sum()/df_obj.shape[0])) ## missing values percentage 
        c=c+1
print(c)

- Missing values have been filled in df_obj

#### But there is a need to clean the following columns in order to use them appropriately for modelling
- Power
- ARAI_Certified_Mileage
- Ground_Clearance
- Bootspace
- Torque

#### 6 Power

##### first fetching 4 characters from it then correct it.

In [ ]:
df_obj["Power"]=df_obj["Power"].apply(lambda x:x[0:4])
#df_obj["Power"].unique()

In [ ]:
l1=[]
for i in df_obj["Power"]:
    s1=""
    for j in i:
        if j=="u" and "n" and "d" and "e" and "f" and "i" and "n" and "e" and "d":
            #print(j)
            s1="undefined"
        elif j.isdigit() or j==".":
            s1=s1+j
    #print(s)
    l1.append(s1)

In [ ]:
df_obj["Power"]=l1
#df_obj["Power"].unique()

#### 7 ARAI_Certified_Mileage

#### First check the unique values

In [ ]:
df_obj["ARAI_Certified_Mileage"].unique()

##### Splitting the column on the basis of " " and then removing "kmpl".

In [ ]:
df_obj["ARAI_Certified_Mileage"]= df_obj["ARAI_Certified_Mileage"].str.split(' ', expand=True)[0]

In [ ]:
df_obj["ARAI_Certified_Mileage"]=df_obj["ARAI_Certified_Mileage"].str.replace("kmpl","")

##### Mileage cannot be in four digit of any car so this is an ambiguity, we will correct 1449 by replacing it with 14.49.

In [ ]:
df[(df["ARAI_Certified_Mileage"]=='1449 km/litre')]

##### The entry is found at index number 1036 so we correct it with 14.49:

In [ ]:
df.loc[1036,"ARAI_Certified_Mileage"]

In [ ]:
df.loc[1036,"ARAI_Certified_Mileage"]=14.49

In [ ]:
df.loc[1036,"ARAI_Certified_Mileage"]

- The entry has been rectified.

In [ ]:
#df_obj["ARAI_Certified_Mileage"].unique()

#### 8 Torque

In [ ]:
#df_obj["Torque"].unique()

##### Splitting the column on the basis of "@" and then correcting it.

In [ ]:
df_obj["Torque"]=df_obj["Torque"].str.split('@', expand=True)[0]

In [ ]:
l2=[]
for i in df_obj["Torque"]:
    s=""
    for j in i:
        #print(j)
        if j=="u" and "n" and "d" and "e" and "f" and "i" and "n" and "e" and "d":
            #print(j)
            s="undefined"
        elif j.isdigit() or j==".":
            s=s+j
    #print(s)
    l2.append(s)
df_obj["Torque"]=l2

In [ ]:
#df_obj["Torque"].unique()

#### 9 Ground_Clearance

In [ ]:
#df_obj["Ground_Clearance"].unique()

##### Removing " mm" from Ground_Clearance column

In [ ]:
df_obj["Ground_Clearance"]=df_obj["Ground_Clearance"].str.replace(" mm","")

In [ ]:
#df_obj["Ground_Clearance"].unique()

#### 10 Boot_Space

In [ ]:
#df_obj["Boot_Space"].unique()

##### Removing " litres" from Boot_Space column.

In [ ]:
df_obj["Boot_Space"]=df_obj["Boot_Space"].str.replace(" litres","")

In [ ]:
df_obj["Boot_Space"].replace('209(All3RowsUp).550(3rdRowFolded)&803(2ndRowand3rdRowFolded)','209',inplace=True)

In [ ]:
#df_obj["Boot_Space"].unique()

#### Will change type of all columns together later

In [ ]:
for i in df_obj.columns:
    if df_obj[i].isnull().sum()>0:
        if "Yes" in df_obj[i].unique():
            df_obj[i].fillna("not_defined",inplace=True)
        elif i in ["Torque","ARAI_Certified_Mileage","Boot_Space","Displacement","Ground_Clearance"]:
            df_obj[i].fillna("undefined",inplace=True)
        else:
            df_obj[i]=df_obj[i].fillna(df_obj[i].mode()[0])
df_obj.isnull().sum().sum()


In [ ]:
for i in df_obj.columns:
    print(i,"---------",df_obj[i].unique())

#### 2.6.6 Correction 
- Bcz of this "9.8-10.0", we are not able to convert data type to float, so to resolve this we will take the average of this.

In [ ]:
i="9.8-10.0"
l=i.split("-")
l[0]=float(l[0])
l[1]=float(l[1])
a=(l[0]+l[1])/2
print(a)
#df_obj[i]= df_obj[i].str.split("-", expand=True)[0] 
#for i in df_obj["Price"]:
    #for j in i:
        #if "-" in i:

In [ ]:
l=["Torque","ARAI_Certified_Mileage","Ground_Clearance","Boot_Space","Displacement"]
for j in l:
    #print(j)
    for i in df_obj[j]:
        #print(i)
        if "-" in i:
            l1=i.split("-")
            print(l1)
            l1[0]=float(l1[0])
            l1[1]=float(l1[1])
            avg=(l1[0]+l1[1])/2
            print(avg)
            avg=str(avg)
            print(type(avg))
            df_obj[j]=df_obj[j].replace(i,avg)
            print(df_obj[j])

#### Checking the unique values again for any ambiguity

In [ ]:
for i in df_obj.columns:
    print(i,"---------",df_obj[i].unique())

####  2.6.7 Making null again and filling

In [ ]:
l=["Torque","Power","ARAI_Certified_Mileage","Boot_Space","Price","Displacement","Ground_Clearance"]
for i in l:
    df_obj[i]=df_obj[i].replace("undefined",np.nan)

#### 2.6.8 Converting below mentioned columns of df_obj having numeric columns into float type

In [ ]:
for column in["Torque","Power","ARAI_Certified_Mileage","Boot_Space","Price","Displacement","Ground_Clearance"]:
    df_obj[column]=df_obj[column].astype("float64")

##### 2.6.9 Filling the NaN with mean.

In [ ]:
for column in["Torque","ARAI_Certified_Mileage","Boot_Space","Displacement","Ground_Clearance"]:
    df_obj[column] =df_obj[column].fillna(df_obj[column].mean())

In [ ]:
df_obj.isnull().sum().sum()

#### 2.6.10 Joining the two dataframes df_numeric and df_obj into df

In [ ]:
df=pd.concat([df_numeric,df_obj],axis=1)
df.shape

#### Viewing the top 3 rows of df

In [ ]:
df.head(3)

#### Cylinders and Valves_Per_Cylinder are same so dropping one

In [ ]:
df.drop(columns=["Valves_Per_Cylinder"],inplace=True)

In [ ]:
df.shape

## Step 3 :  Data Insights
- Based on target (Price)

#### How many cars of each companies ?

In [ ]:
plt.figure(figsize=(15,15))
sns.countplot(x="Make",data=df)
plt.xticks(rotation=90);

##### Insight:
- Maruti suzuki has highest number of cars

- Hyundai has second number of highest cars

- DC has less number of cars

#### Min,Max, and Average price 

In [ ]:
df["Price"].agg(["min","max","mean"])

#### Insight :
- Minimum price : 2.364470e+05 (i.e 2 lakhs 36 thousands)

- Maximum Price : 2.121554e+08 (i.e 21 crores)

- Average Price : 4.596538e+06 ( i.e 45 lakhs)

#### Companies whose price is greater than avg price of the car

In [ ]:
df[df["Price"]>np.mean(df["Price"])][["Price","Model","Make"]].sort_values(by="Price",ascending=False)

#### Insight :
- 107 cars having their price is greater than average price i.e 4.596538e+06 and above are the names of those cars.
- Bugatti has the highest above average price i.e. 212155397

In [ ]:
sns.distplot(df["Price"])

####  Detail of car having max price

In [ ]:
df[df["Price"]==df["Price"].max()]

#### Insight :
- Chiron of bugatti company was the most expensive car with above mentioned features.

#### Detail of car having min price

In [ ]:
df[df["Price"]==df["Price"].min()]

#### Insight :
- Cheapest car was Nano Genx of company Tata with above features details

In [ ]:
len(df[df["Price"]>=df["Price"].mean()])

#### 3.1 Saving the data of above average price in df_a 

In [ ]:
df_a=df[df["Price"]>=df["Price"].mean()][["Make","Model","Variant","Price"]]
df_a

#### No of companies having price greater than average price

In [ ]:
df_a["Make"].nunique()

##### Insight:
-  above 20 companies' cars price is greater than average price.

#### Name of companies having price greater than average price

In [ ]:
df_a["Make"].value_counts().sort_values(ascending=False)

- Mercedes-Benz have the highest above average price then Land Rover and Bmw.

#### Find out the average price of car whose price is greater than average price of the cars in the data

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(x="Make",y="Price",data=df_a)
plt.xticks(rotation=90);

##### Insight :
- Most Expensive company's car :
                                - 1. Bugati
                                - 2. Rolls-Royace
                                - 3. Ferrari

- Least Expensive company's car : Mitsubishi

#### Average price of no of cars whose price is less than avg car price

In [ ]:
len(df[df["Price"]<=df["Price"].mean()])

In [ ]:
sns.boxplot(df["Price"])

- Variation in Price bcz of outliers.

In [ ]:
df[df["Price"]>=150000000]

- Extreme outliers, we will do analysis without these two cars.

#### 3.2 Saving the data without extreme outliers in x

In [ ]:
x=df[df["Price"]<=150000000]
x

In [ ]:
x["Price"].mean()

#### 3.3 Correlation of numeric columns w.r.t. price

In [ ]:
df.corr()["Price"]

#### Insight:
- Cylinders, Displacement, Power, and Torque are positively correlated.
- Seating_Capacity is negatively correlated.

###### Cylinders,Displacement,Power and Torque are considered important features with respect to car price.

#### 3.4 Taking numeric columns and see the relation with Price using scatter plot

In [ ]:
numeric=["int32","float64"]
n=1
plt.figure(figsize=(20,20))
for i in x.select_dtypes(include=numeric):
    if i!="Price":
        plt.subplot(4,3,n)
        plt.scatter(x[i],x["Price"])
        #plt.title(i)
        plt.xlabel(i)
        plt.ylabel("Price")
        n=n+1

#### Insight:
- Price, Cylinders, Doors, and Seating_Capacity(mostly 4-5 seats cars are costlier) don't have linear relationship
- Displacement, Power, and Torque have sort of linear relationship

#### How many cars of each company?

In [ ]:
df["Make"].value_counts()

#### Insight:
- Most Demanding companies are Maruti Suzuki, Hyundai, Mahindra, Tata, Toyota.
- Least in Demand is Dc

#### Min, Max, and avg Price of the cars based on their company

In [ ]:
df.groupby(["Make"])["Price"].agg(["min","max","mean"]).sort_values(by="mean",ascending=False).head()

#### Insight:
- Top 3 expensive car brands are Bugatti, Rolls-Royce and Ferrari.
- Least expensive car is Bajaj.

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(x="Make",y="Price",data=df);
plt.xticks(rotation=90);

#### Insight :

###### Top 3 Expensive Company's cars average price :
- 1- Bugatti

- 2- Rolls- Royce

- 3- Ferari

##### Cheapest company's car average price:
- Datsun

#### Details of Top 3 expensive companies and their cars

In [ ]:
df[((df["Make"]=="Bugatti")|(df["Make"]=="Ferrari")|(df["Make"]=="Rolls-Royce"))].head()

#### Now we will find out why Bugatti, Rolls-Royce, and Ferrari are expensive cars, what features they have provided so we compare these on the basis of some premium features based on domain knowledge .

#### 3.5 We will consider following premium features in order to differentiate the above top 3 expensive cars with respect to price:
- Premium feeatures which makes the difference of price in above premium cars(top 3 expensive cars)

- CYLINDER
- DISPLACEMENT
- POWER
- TORQUE


In [ ]:
grp=df.groupby(["Make"])["Cylinders","Displacement","Power","Torque"]

In [ ]:
grp.get_group("Bugatti")

- Bugatti has 16 cylinders and highest Displacement, Power and Torque, features for high cost.

In [ ]:
grp.get_group("Rolls-Royce")

- Rolls-Royce has 12 cylinders and second highest Displacement, Power and Torque.

In [ ]:
grp.get_group("Ferrari")

- Ferrari has 8 and 12 cylinders and third highest Displacement, Power and Torque.

#### Insight :
- From above output we can see no.of cylinders in buggati is maximum and in rolls royce and ferrari none of the above premium feature clearly differentiate them so based on other features we can say their price is different since ferrari is top brand sport car and Rolls Royce is the epitome of comfort so their type is totally different.

#### 3.6 Buyers can choose the car based on the following important features:
- 1. safety---- no of airbags
- 2. power
- 3. torque 
- 4. mileage
- 5. seating_capacity and 
- 6. fuel_type.

#### 1 Number_of_Airbags

#### Maximum,minimum and average air bags a company can give

In [ ]:
df.groupby(["Make"])["Number_of_Airbags"].agg(["min","max","mean"]).sort_values(by="mean",ascending=False).head()

#### Insight :
###### According to safety point of view top 3 companies :

- 1- Lexus

- 2- Audi

- 3- land rover

###### Found to be good company cars which provides average 10.6, 7.58 , 7.48 number of air bags repectively.

#### 2 Power

#### Max,min and average power a company can give

In [ ]:
df.groupby(["Make"])["Power"].agg(["min","max","mean"]).sort_values(by="mean",ascending=False).head()

#### Insight :
##### According to Power point of view top 4 companies :

- 1- Bugatti

- 2- Ferrari

- 3- Lamborghini

- 4- Bentley

###### Found to be good company cars which provides average power 1539.5, 656.75, 636.46 , 563.83 repectively.

#### 3 Torque

#### Max,min and average Torque of each company 

In [ ]:
df.groupby(["Make"])["Torque"].agg(["min","max","mean"]).sort_values(by="mean",ascending=False).head()

#### Insight :
##### According to Torque point of view top 4 companies :

- 1- Bugatti

- 2- Bentley

- 3- Rolls-Royce

- 4- Ferrari

###### Found to be good company cars which provides average torque 1539.5, 811.6, 763.33, 718.75  repectively.

#### 4 Seating_Capacity

#### Max,min and average seating capacity of each company

In [ ]:
df.groupby(["Make"])["Seating_Capacity"].agg(["min","max","mean"]).sort_values(by="mean",ascending=False).head()

#### Insight :
##### According to seating_capacity point of view top 3 companies :

- 1- Icml : 8 seater

- 2- Mitsubishi : 7 seater

- 3- Mahindra : 6 seater

####  5 ARAI_Certified_Mileage	

#### Max,min and average Mileage of each company 

In [ ]:
df.groupby(["Make"])["ARAI_Certified_Mileage"].agg(["min","max","mean"]).sort_values(by="mean",ascending=False).head()

#### From above we can see The company Mercedes-Benz gives maximum mileage : 142 which is not possible there might be typing mistake it might be 14.2

In [ ]:
df[(df["ARAI_Certified_Mileage"]==142.0)]

- The entries are found at index number 794,795,799 and 800 so we correct it with 14.2:

In [ ]:
df.loc[794,"ARAI_Certified_Mileage"]=14.2
df.loc[795,"ARAI_Certified_Mileage"]=14.2
df.loc[799,"ARAI_Certified_Mileage"]=14.2
df.loc[800,"ARAI_Certified_Mileage"]=14.2

In [ ]:
df.loc[794,"ARAI_Certified_Mileage"]

- The entries have been rectified now compute the max,min average again.

In [ ]:
df.groupby(["Make"])["ARAI_Certified_Mileage"].agg(["max","min","mean"]).sort_values(by="mean",ascending=False)

#### Insight :
##### According to ARAI_Certified_Mileage point of view top 3 companies :

- 1- Bajaj

- 2- Maruti Suzuki

- 3- Honda

###### Found to be good company cars which provides average ARAI_Certified_Mileage 29, 22.5 , 21.3 repectively.

#### 6 Fuel_type

In [ ]:
df["Fuel_Type"].value_counts()

- Customers preferred Petrol fuel_type.

In [ ]:
g_f=df.groupby(["Make"])["Fuel_Type"]

In [ ]:
#for i in df["Make"]:
    #print(i,"\n",g_f.get_group(i).values)

#### 7 Extra Features

#### Taking those  features having only two unique values (yes or no) and we will treat them as extra features.

In [ ]:
extra_features = ["Navigation_System","Second_Row_AC_Vents","ESP_(Electronic_Stability_Program)","Cooled_Glove_Box","Turbocharger","Rain_Sensing_Wipers","Automatic_Headlamps","Cruise_Control","Make","Price","EBA_(Electronic_Brake_Assist)","EBD_(Electronic_Brake-force_Distribution)","ABS_(Anti-lock_Braking_System)","Gear_Indicator","Auto-Dimming_Rear-View_Mirror","Hill_Assist","Key_Off_Reminder","USB_Compatibility","Model"]
extra_features

#### df2 having extra features with company name and price

In [ ]:
df2=df[extra_features]
df2  

#### We will compare with Model not Make

##### Making copy of df2 and save it to df3 having extra features and setting model as index.

In [ ]:
df3=df2.copy()
df3.set_index("Model",inplace=True)
df3

#### Zipping of model name with their corresponding column values

In [ ]:
for i,j in zip(df3.index,df3.values):
    print(i,"-------------",j)

#### Counting the number of yes features a model has along with its price.

In [ ]:
for i,j in zip(df3.index,df3.values):
    c=0
    l=[]
    for k in j:
        if type(k)==str:
            if k=="Yes":
                c=c+1
                l.append(k)
    print("Model--------",i,",--------Count------",c,",---------Price-------",j[9])

#### Insight :
###### From above output we can see price increases with increase number of features and those models which have different price on same number of features where other factors like torque , power matters like nano genx cars' etc.


###### Not much of a relation of Price with these features as we can see, with some model price increases and with some model its decreases.

## Step 4 : Data Preparation

In [ ]:
df.shape

#### We will use Mean Target encoding,and its drawback is Data Leakage(Overfitting) and for overfitting we will use cross validation.

#### 4.1 Dropping irrelevant features from ML POV.

In [ ]:
df.drop(columns=["Make","Tachometer","Tripmeter","Front_Suspension","Rear_Suspension","Audiosystem","Third_Row_AC_Vents","Ventilation_System","Key_Off_Reminder","Cooled_Glove_Box","Gear_Shift_Reminder","Rain_Sensing_Wipers","Seat_Height_Adjustment","Adjustable_Steering_Column","Rear_Center_Armrest"],inplace=True)

In [ ]:
df.shape

#### 4.2 Encoding
- Label Encoding for Yes/NO values:

In [ ]:
l = ['Navigation_System', 'Second_Row_AC_Vents',
       'ESP_(Electronic_Stability_Program)',
       'Turbocharger','Automatic_Headlamps',
       'Cruise_Control','EBA_(Electronic_Brake_Assist)',
       'EBD_(Electronic_Brake-force_Distribution)',
       'ABS_(Anti-lock_Braking_System)','Gear_Indicator',
       'Auto-Dimming_Rear-View_Mirror', 'Hill_Assist',
       'USB_Compatibility',"ASR_/_Traction_Control"]
for i in l:
    df[i].replace({"Yes":1,"not_defined":0},inplace=True)
df.head(3)

#### 4.2.1 ENCODING of the remaining columns

In [ ]:
df["Drivetrain"].unique()

In [ ]:
df["Drivetrain"]=df["Drivetrain"].replace({"RWD (Rear Wheel Drive)":0,"FWD (Front Wheel Drive)":1,"AWD (All Wheel Drive)":2,"4WD":3})

In [ ]:
df["Emission_Norm"].unique()

In [ ]:
df["Emission_Norm"]=df["Emission_Norm"].replace("BS 6","BS VI")

In [ ]:
df["Emission_Norm"]=df["Emission_Norm"].replace({'BS III':0,'BS IV':1, 'BS VI':2})

In [ ]:
df["Keyless_Entry"].unique()

In [ ]:
df["Keyless_Entry"]=df["Keyless_Entry"].replace("Remote, Smart Key","Smart Key, Remote")

In [ ]:
df["Keyless_Entry"]=df["Keyless_Entry"].replace({'not_defined':0,'Remote':1, 'Yes':2, 'Smart Key':3,'Smart Key, Remote':4})

In [ ]:
df["Odometer"].unique()

In [ ]:
df["Odometer"]=df["Odometer"].replace({'not_defined':0,'Digital':1, 'Analog':2, 'Digital, Analog':3, 'Yes':4})

In [ ]:
df["Speedometer"].unique()

In [ ]:
df["Speedometer"]=df["Speedometer"].replace( 'Digital, Analog','Analog, Digital')

In [ ]:
df["Speedometer"]=df["Speedometer"].replace({'Analog':0, 'Digital':1, 'Analog, Digital':2,'not_defined':3, 'Yes':4})

In [ ]:
df["Fuel_Type"]=df["Fuel_Type"].replace({"Petrol":0,"CNG":1,"Diesel":2,"CNG + Petrol":3,"Hybrid":4,"Electric":5})

In [ ]:
df["Parking_Assistance"].unique()

In [ ]:
df["Parking_Assistance"]=df["Parking_Assistance"].replace({"Rear sensors, Rear sensors with camera":"Rear sensors with camera","Rear sensors with camera, Rear sensors":"Rear sensors with camera","Front sensors, Rear sensors":"Rear sensors, Front sensors","Front and rear sensors with camera, Front & rear sensors with 360 degree view":"Front & rear sensors with 360 degree view"})

In [ ]:
df["Parking_Assistance"]=df["Parking_Assistance"].replace({'not_defined':0, 'Rear sensors':1, 'Rear sensors with camera':2,
                                                           'Front and rear sensors with camera':3,
                    'Yes':4, 'Front & rear sensors with 360 degree view':5,'Rear sensors, Front sensors':6})

In [ ]:
df.shape

#### 4.2.2 We will apply Mean Target Encoding on Model and Variant.

In [ ]:
df2=df.groupby(["Model"])["Price"].agg(["mean"]).reset_index()
df2.head()

##### Model

In [ ]:
l=[]
for i in df["Model"]:
    for j,k in df2.iterrows():
        if i==k["Model"]:
            l.append(k["mean"])

In [ ]:
l

In [ ]:
df["Model"]=l
df.head()

#### Variant

In [ ]:
df2=df.groupby(["Variant"])["Price"].agg(["mean"]).reset_index()
df2.head()

In [ ]:
l=[]
for i in df["Variant"]:
    for j,k in df2.iterrows():
        if i==k["Variant"]:
            l.append(k["mean"])

In [ ]:
l

In [ ]:
df["Variant"]=l
df.head()

## Step 5 : Data Modelling and Evaluation